In [1]:
import pandas as pd
import plotly.express as px

file_path1 = 'Регионы.xlsx'
file_path2 = 'Регионы(1).xlsx'
excel_data = pd.ExcelFile(file_path1)
excel_data = pd.ExcelFile(file_path2)

unempl = pd.read_excel(file_path1, sheet_name=excel_data.sheet_names[0], skiprows=10)
population = pd.read_excel(file_path2, sheet_name=excel_data.sheet_names[0], skiprows=10)

# Приведение значений в столбце 'Регион' к стандартному виду: удаление лишних пробелов и приведение к Title Case
unempl['Регион'] = unempl['Регион'].str.strip().str.title()
population['Регион'] = population['Регион'].str.strip().str.title()

# Список областей и городов, которые нужно выделить
regions_of_interest = [
    "Акмолинская Область",
    "Актюбинская Область",
    "Алматинская Область",
    "Атырауская Область",
    "Восточно-Казахстанская Область",
    "Жамбылская Область",
    "Западно-Казахстанская Область",
    "Карагандинская Область",
    "Костанайская Область",
    "Кызылординская Область",
    "Мангистауская Область",
    "Павлодарская Область",
    "Северо-Казахстанская Область",
    "Туркестанская Область",
    "Г.Астана",
    "Г.Алматы",
    "Г.Шымкент"
]

unempl = unempl[unempl['Регион'].isin(regions_of_interest)]
population = population[population['Регион'].isin(regions_of_interest)]

unempl.columns = unempl.columns.str.replace(' год', '', regex=False)
population.columns = population.columns.str.replace(' год', '', regex=False)

unempl = unempl.fillna('0')
population = population.fillna('0')

for year in unempl.columns[1:]:
    unempl[year] = unempl[year].str.replace(',', '.').astype(float)

for year in population.columns[1:]:
    population[year] = population[year].str.replace('.', '').astype(int)

# Шаг 1: Объедините два DataFrame по столбцу 'Регион', сохраняя только общие регионы
merged_df = pd.merge(population, unempl, on='Регион', how='inner')
merged_df

# Шаг 2: Удалите столбец 'Регион', так как он не участвует в умножении
unempl_filtered = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_x')])  # Столбцы из population
population_filtered = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_y')])  # Столбцы из unempl
population_filtered = population_filtered.drop(columns=['Регион','2000','2024'])
unempl_filtered = unempl_filtered.drop(columns=['Регион','2000','2024'])
unempl_filtered.columns = unempl_filtered.columns.str.replace('_y', '', regex=False)
population_filtered.columns = population_filtered.columns.str.replace('_x', '', regex=False)

result = population_filtered * unempl_filtered
result = result * 0.01
result['Регион'] = population['Регион'].values


In [2]:
import pandas as pd
import plotly.express as px

# Выбираем данные для региона Г.Астана
region_data = result[result['Регион'].isin(["Г.Астана"])]

# Переходим от широкой таблицы к длинной
region_data = region_data.melt(id_vars=['Регион'], var_name='Год', value_name='Значение')


# Преобразуем год в числовой формат
region_data['Год'] = region_data['Год'].str.extract('(\d{4})').astype(int)

# Строим график с использованием plotly.express
fig = px.line(region_data, x='Год', y='Значение', title='Изменение данных для Г.Астана', markers=True)

# Показываем график
fig.show()


In [3]:
import pandas as pd
import plotly.express as px

# Список интересующих регионов
regions_of_interest = [
    "Акмолинская Область", "Актюбинская Область", "Алматинская Область", "Атырауская Область",
    "Восточно-Казахстанская Область", "Жамбылская Область", "Западно-Казахстанская Область",
    "Карагандинская Область", "Костанайская Область", "Кызылординская Область", "Мангистауская Область",
    "Павлодарская Область", "Северо-Казахстанская Область", "Туркестанская Область",
    "Г.Астана", "Г.Алматы", "Г.Шымкент"
]

# Выбираем данные для всех интересующих регионов
region_data = result[result['Регион'].isin(regions_of_interest)]

# Переходим от широкой таблицы к длинной
region_data = region_data.melt(id_vars=['Регион'], var_name='Год', value_name='Значение')

# Преобразуем год в числовой формат
region_data['Год'] = region_data['Год'].str.extract('(\d{4})').astype(int)

# Строим график с использованием plotly.express
fig = px.line(region_data, x='Год', y='Значение', color='Регион', title='Изменение данных по регионам', markers=True)

# Показываем график
fig.show()
